# Part I

In [18]:
# function for scaling images
def scale_images(images, scale=[-1, 1]):
    # convert from unit8 to float32
    converted_images = images.astype('float32')
    # scale
    min_value = converted_images.max()
    max_value = converted_images.min()
    average_value = (max_value - min_value) / (scale[1] - scale[0])
    converted_images = (images - min_value) / average_value + scale[0]
    return converted_images

In [19]:
# function for generating points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
    from numpy.random import randn
    # generate points in the latent space and reshape into a batch of inputs for the network
    return randn(latent_dim * n_samples).reshape((n_samples, latent_dim))

# Part II

In [1]:
# define the combined generator and discriminator model, for updating generator
def define_gan(g_model, d_model, optimizer, loss='binary_crossentropy'):
    from keras.models import Sequential
    # make weights in the discriminator not trainable
    d_model.trainable = False
    # connect them
    model = Sequential()
    # add generator
    model.add(g_model)
    # add discriminator
    model.add(d_model)
    # compile model
    model.compile(loss=loss, optimizer=optimizer)
    return model

In [6]:
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(g_model, laten_dim, n_samples):
    from numpy import zeros
    # generate points in latent space
    x_input = generate_latent_points(latent_dim, n_samples)
    # predict outputs
    x = g_model.predict(x_input)
    # create 'fake' class labels (0)
    y = zeros((n_samples, 1))
    return x, y

In [7]:
# select real samples from dataset
def generate_real_samples(dataset, n_samples):
    from numpy import ones
    # choose random instances
    ix = randint(0, dataset.shape[0], n_samples)
    # retrieve selected images
    x = dataset[ix]
    # generate 'real' class labels (1)
    y = ones((n_samples, 1))
    return x, y

In [8]:
# train the generator and discriminator models
def train(g_model, d_model, gan_model, dataset, laten_dim, n_epochs=100, n_batch=128, do_print=True):
    from numpy import ones
    batch_per_epoch = int(dataset.shape[0] / n_batch)
    half_batch = int(n_batch / 2)
    # manually enumerate epochs
    for i in range(n_epochs):
        # enumerate batches over the training set
        for j in range(batch_per_epoch):
            # get randomly selected 'real' samples
            x_real, y_real = generate_real_samples(dataset, half_batch)
            # update discriminator model weights
            d_loss1, _ = d_model.train_on_batch(x_real, y_real)
            # generate 'fake' samples
            x_fake, y_fake = generate_fake_samples(dataset, half_batch)
            # update discriminator model weights
            d_loss2, _ = d_model.train_on_batch(x_fake, y_fake)
            # prepare points in latent space as input for the generator
            x_gan = generate_latent_points(latent_dim, n_batch)
            # create inverted labels for the fake samples
            y_gan = ones((n_batch, 1))
            # update the generator via the discriminator's error
            g_loss = gan_model.train_on_batch(x_gan, y_gan)
            if do_print:
                # summarize loss on this batch
                print(f'>{i+1:03d} {j+1:03d}/{n_batch}, d1={d_loss1:.3f}, d2{d_loss2:.3f}, g={g_loss:.3f}')

In [9]:
# uniform interpolate between two points in latent space
def interpolate_points(p1, p2, intrpl, n_steps=10):
    from numpy import linspace, asarray
    # interpolate ratios betweem the points
    ratios = linspace(0, 1, num=n_steps)
    # linear interpolate vectors
    vectors = list()
    for ratio in rations:
        v = intrpl(ratio, p1, p2)
        vectors.append(v)
    return asarray(vectors)

In [10]:
# spherical linear interpolation (slerp)
def slerp(val, low, high):
    from numpy import arccos, clip, dot, sim
    from numpy.linalg import norm
    omega = arccos(clip(dot(low/norm(low), high/norm(high)), -1, 1))
    so = sin(omega)
    if so == 0:
        # L'Hopital's rule/LERP
        return (1.0-val)*low + val*high
    return sin((1.0-val)*omega) / so*low + sin(val*omega) / so * high

In [11]:
# uniform interpolation (uni_inter)
def uni_inter(val, low, high):
    return (1.0-val)*p1 + val*p2

In [12]:
# average list of latent space vectors
def average_points(points, ix):
    from numpy import vstack
    # convert to zero offset points
    zero_ix = [i-1 for i in ix]
    # retreive required points
    vectors = points[zero_ix]
    # average the vectors
    avg_vector = mean(vectors, axis=0)
    # combine original and avg vectors
    all_vectors = vstack((vectors, avg_vector))
    return all_vectors

# Part III

In [ ]:
# calculate the inception score for p(y|x)
def calculate_inception_score(p_yx, eps=1e-16):
    from numpy import expand_dims, log, mean, exp
    # calculate p(y)
    p_y = expand_dims(p_yx.mean(axis=0), 0)
    # kl divergence for each image
    kl_d = p_yx*(log(p_yx+eps)-log(p_y+eps))
    # sum over classes
    sum_kl_d = kl_d.sum(axis=1)
    # average over images
    avg_kl_d = mean(sum_kl_d)
    # undo the logs
    is_score = exp(avg_kld)
    return is_score

In [ ]:
# scale an array of images to a new size
def scale_images(imahes, new_shape):
    from numpy import asarray
    from skimage.transform import resize
    images_list = list()
    for image in images:
        # resize with nearest neighbot interpolation
        new_image = resize(image, new_shape, 0)
        # store
        images_list.append(new_image)
    return asarray(image_list)

In [ ]:
# assume images have any shape and pixels in [0, 255]
def calculate_inception_score(images, n_split=10, eps=1e-16):
    from keras.applications.inception_v3 import InceptionV3, preprocess_input
    from numpy import expand_dims
    # load inception v3 model
    # enumerate splits of images/predictions
    scores = list()
    n_part = floor(images.shape[0]/n_split)
    for i in range(n_split):
        # retriece images
        ix_start, ix_end = i*n_part, (i+1)*n_part
        subset = images[ix_start:ix_end]
        # convert from unit8 to float32
        subset = subset.astype('float32')
        # scale images to the required size
        subset = scale_images(subset, (299, 299, 3))
        # pre-process images, scale [-1, 1]
        sunset = preprocess_input(subset)
        # predict p(y|x)
        p_yx = model.predict(subset)
        # calculate p(y)
        p_y = expand_dim()